In [1]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

In [2]:
conn = psycopg2.connect(dbname='demo', user='popov', password='*****', host='localhost')

In [3]:
query = """SELECT * FROM bookings.aircrafts;"""

aircrafts = pd.read_sql(query, conn)
aircrafts.head()

,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
2,SU9,Sukhoi SuperJet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600


##  Выборки

Для начала поставим перед собой такую задачу: выбрать все самолеты компании Airbus. В этом нам поможет оператор поиска шаблонов LIKE:

```SQL
SELECT * FROM aircrafts WHERE model LIKE 'Airbus%';
```

In [6]:
aircrafts[aircrafts.model.str.contains('Airbus')]

,aircraft_code,model,range
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700


Cуществует и оператор `NOT LIKE`. Например, если мы захотим узнать, каки- ми самолетами, кроме машин компаний Airbus и Boeing, располагает наша авиаком- пания, то придется усложнить условие:

```SQL
SELECT * FROM aircrafts
  WHERE model NOT LIKE 'Airbus%'
    AND model NOT LIKE 'Boeing%';
```

In [12]:
aircrafts[~aircrafts.model.str.contains('Airbus|Boeing')]

,aircraft_code,model,range
2,SU9,Sukhoi SuperJet-100,3000
7,CN1,Cessna 208 Caravan,1200
8,CR2,Bombardier CRJ-200,2700


Кроме символа «%» в шаблоне может использоваться и символ подчеркивания — «\_», который соответствует в точности одному любому символу. В качестве примера найдем в таблице «Аэропорты» те из них, которые имеют названия длиной три символа (буквы). С этой целью зададим в качестве шаблона строку, состоящую из трех символов «\_».

```SQL
SELECT * FROM airports WHERE airport_name LIKE '___';
```

In [4]:
query = """SELECT * FROM bookings.airports;"""

airports = pd.read_sql(query, conn)
airports.head()

,airport_code,airport_name,city,longitude,latitude,timezone
0,MJZ,Мирный,Мирный,114.038928,62.534689,Asia/Yakutsk
1,NBC,Бегишево,Нижнекамск,52.060000,55.340000,Europe/Moscow
2,NOZ,Спиченково,Новокузнецк,86.877200,53.811400,Asia/Novokuznetsk
3,NAL,Нальчик,Нальчик,43.636600,43.512900,Europe/Moscow
4,OGZ,Беслан,Владикавказ,44.606600,43.205100,Europe/Moscow


In [14]:
airports[airports['airport_name'].str.len() == 3]

,airport_code,airport_name,city,longitude,latitude,timezone
58,UFA,Уфа,Уфа,55.874417,54.557511,Asia/Yekaterinburg


Существует ряд операторов для работы с регулярными выражениями POSIX. Эти операторы имеют больше возможностей, чем оператор `LIKE`. Для того чтобы выбрать, например, самолеты компаний Airbus и Boeing, можно сделать так:

```SQL
SELECT * FROM aircrafts WHERE model ~ '^(A|Boe)';
```

In [15]:
aircrafts[aircrafts.model.str.contains(r'^(A|Boe)')]

,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700
6,733,Boeing 737-300,4200


Для инвертирования смысла оператора `~` нужно перед ним добавить знак «!». В качестве примера отыщем модели самолетов, которые не завершаются числом 300.

```SQL
SELECT * FROM aircrafts WHERE model !~ '300$';
```
В этом регулярном выражении символ «$» означает привязку поискового шаблона к концу строки. Если же требуется проверить наличие такого символа в составе строки, то перед ним нужно поставить символ обратной косой черты «\».

In [49]:
aircrafts[aircrafts.model.str.contains(r'^(?!.*300).*$')]

,aircraft_code,model,range
2,SU9,Sukhoi SuperJet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700
7,CN1,Cessna 208 Caravan,1200
8,CR2,Bombardier CRJ-200,2700


В качестве замены традиционных операторов сравнения могут использоваться __предикаты сравнения__, которые ведут себя так же, как и операторы, но имеют другой синтаксис.
Давайте ответим на вопрос: какие самолеты имеют дальность полета в диапазоне от 3 000 км до 6 000 км? Ответ получим с помощью предиката `BETWEEN`.

```SQL
SELECT * FROM aircrafts WHERE range BETWEEN 3000 AND 6000;
```

In [50]:
aircrafts[aircrafts.range.between(3000, 6000)]

,aircraft_code,model,range
2,SU9,Sukhoi SuperJet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
6,733,Boeing 737-300,4200


Обратите внимание, что граничное значение 3 000 включено в полученную выборку. Чтобы исключить границы значений, нужно добавить параметр `inclusive=False`.

In [51]:
aircrafts[aircrafts.range.between(3000, 6000, inclusive=False)]

,aircraft_code,model,range
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
6,733,Boeing 737-300,4200


При выборке данных можно проводить вычисления и получать в результирующей таблице **вычисляемые столбцы**. Если мы захотим представить дальность полета не только в километрах, но и в милях, то нужно вычислить это выражение и для удобства присвоить новому столбцу псевдоним с помощью ключевого слова `AS`.

```SQL
SELECT model, range, range / 1.609 AS miles FROM aircrafts;
```

In [55]:
aircrafts[['model', 'range']].assign(miles = aircrafts.range / 1.609)

,model,range,miles
0,Boeing 777-300,11100,6898.694842
1,Boeing 767-300,7900,4909.881914
2,Sukhoi SuperJet-100,3000,1864.512119
3,Airbus A320-200,5700,3542.573027
4,Airbus A321-200,5600,3480.422623
5,Airbus A319-100,6700,4164.077067
6,Boeing 737-300,4200,2610.316967
7,Cessna 208 Caravan,1200,745.804848
8,Bombardier CRJ-200,2700,1678.060907


По всей вероятности, такая высокая точность представления значений в милях не требуется, поэтому мы можем уменьшить ее до разумного предела в два десятичных знака:

```SQL
SELECT model, range, round(range / 1.609, 2) AS miles FROM aircrafts;
```

In [56]:
aircrafts[['model', 'range']].assign(miles = aircrafts.range / 1.609).round(2)

,model,range,miles
0,Boeing 777-300,11100,6898.69
1,Boeing 767-300,7900,4909.88
2,Sukhoi SuperJet-100,3000,1864.51
3,Airbus A320-200,5700,3542.57
4,Airbus A321-200,5600,3480.42
5,Airbus A319-100,6700,4164.08
6,Boeing 737-300,4200,2610.32
7,Cessna 208 Caravan,1200,745.80
8,Bombardier CRJ-200,2700,1678.06


Обратимся к такому вопросу, как упорядочение строк при выводе. Если не принять специальных мер, то СУБД не гарантирует никакого конкретного порядка строк в результирующей выборке. Для упорядочения строк служит предложение `ORDER BY`, которое мы уже использовали ранее. Однако мы не говорили, что можно задать не только возрастающий, но также и убывающий порядок сортировки. Например, если мы захотим разместить самолеты в порядке убывания дальности их полета, то нужно сделать так:

```SQL
SELECT * FROM aircrafts ORDER BY range DESC;
```

In [59]:
aircrafts.sort_values(by='range', ascending=False)

,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
5,319,Airbus A319-100,6700
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
6,733,Boeing 737-300,4200
2,SU9,Sukhoi SuperJet-100,3000
8,CR2,Bombardier CRJ-200,2700
7,CN1,Cessna 208 Caravan,1200


Мы детально разобрались с таблицей «Самолеты» и теперь обратим наше внимание на таблицу «Аэропорты»). В ней есть столбец «Часовой пояс» (timezone). Давайте посмотрим, в каких различных часовых поясах располагаются аэропорты. Если сделать традиционную выборку

```SQL
SELECT timezone FROM airports;
```
то мы получим список значений, среди которых будет много повторяющихся. Конечно, это неудобно. Для того чтобы оставить в выборке только неповторяющиеся значения, служит ключевое слово `DISTINCT`:

```SQL
SELECT DISTINCT timezone FROM airports ORDER BY 1;
```
Обратите внимание, что столбец, по значениям которого будут упорядочены строки, указан не с помощью его имени, а с помощью его порядкового номера в предложении `SELECT`.

In [66]:
pd.Series(airports.timezone.unique()).sort_values()

16           Asia/Anadyr
12            Asia/Chita
6           Asia/Irkutsk
9         Asia/Kamchatka
5       Asia/Krasnoyarsk
11          Asia/Magadan
2      Asia/Novokuznetsk
13      Asia/Novosibirsk
15             Asia/Omsk
10         Asia/Sakhalin
8       Asia/Vladivostok
0           Asia/Yakutsk
3     Asia/Yekaterinburg
7     Europe/Kaliningrad
1          Europe/Moscow
4          Europe/Samara
14      Europe/Volgograd
dtype: object

Таким образом, аэропорты располагаются в семнадцати различных часовых поясах. Они описаны в базе данных часовых поясов, поддерживаемой международной организацией IANA (Internet Assigned Numbers Authority), и отличаются от традиционных географических и административных часовых поясов, число которых в России равно одиннадцати.

В таблице «Аэропорты» более ста записей. Если мы поставим задачу найти три самых восточных аэропорта, то для ее решения подошел бы такой алгоритм: отсортировать строки в таблице по убыванию значений столбца «Долгота» (longitude) и включить в выборку только первые три строки. Как отсортировать строки по убыванию значений какого-либо столбца, вы уже знаете, а для того чтобы ограничить число строк, включаемых в результирующую выборку, служит предложение `LIMIT`.

```SQL
SELECT airport_name, city, longitude
  FROM airports
  ORDER BY longitude DESC
  LIMIT 3;
```  

In [71]:
airports[['airport_name', 'city', 'longitude']].sort_values('longitude', ascending=False).head(3)

,airport_name,city,longitude
76,Анадырь,Анадырь,177.741483
29,Елизово,Петропавловск-Камчатский,158.453669
33,Магадан,Магадан,150.720439


А как найти еще три аэропорта, которые находятся немного западнее первой тройки, т. е. занимают места с четвертого по шестое? Алгоритм будет почти таким же, как в первой задаче, но он будет дополнен еще одним шагом: нужно пропустить три первые строки, прежде чем начать вывод. Для пропуска строк служит предложение `OFFSET`.

```postgresql
SELECT airport_name, city, longitude
  FROM airports
  ORDER BY longitude DESC
  LIMIT 3
  OFFSET 3;
```

In [75]:
airports[['airport_name', 'city', 'longitude']].sort_values('longitude', ascending=False).iloc[3:6]

,airport_name,city,longitude
30,Хомутово,Южно-Сахалинск,142.717531
78,Хурба,Комсомольск-на-Амуре,136.934000
28,Хабаровск-Новый,Хабаровск,135.188361


В дополнение к вычисляемым столбцам, когда выводимые значения получают путем вычислений, при выборке данных из таблиц можно использовать условные выражения, позволяющие вывести то или иное значение в зависимости от условий.

В таблице «Самолеты» есть столбец «Максимальная дальность полета» (range). Мы можем дополнить вывод данных из этой таблицы столбцом «Класс самолета», имея в виду принадлежность каждого самолета к классу дальнемагистральных, среднемагистральных или ближнемагистральных судов.
Для этого подойдет конструкция

`CASE WHEN условие THEN выражение
     [ WHEN ... ]
     [ ELSE выражение ]
   END
`

Воспользовавшись этой конструкцией в предложении `SELECT` и назначив новому столбцу имя с помощью ключевого слова `AS`, получим следующий запрос:

```postgresql
SELECT model, range,
  CASE WHEN range < 2000 THEN 'Ближнемагистральный'
       WHEN range < 5000 THEN 'Среднемагистральный'
       ELSE 'Дальнемагистральный'
  END AS type
  FROM aircrafts
  ORDER BY model;
```

In [91]:
aircrafts['type'] = aircrafts['range'].apply(lambda x: 'Ближнемагистральный' if x < 2000 else 
                                                    'Среднемагистральный' if x < 5000 else 'Дальнемагистральный')
aircrafts[['model', 'range', 'type']].sort_values('model')

,model,range,type
5,Airbus A319-100,6700,Дальнемагистральный
3,Airbus A320-200,5700,Дальнемагистральный
4,Airbus A321-200,5600,Дальнемагистральный
6,Boeing 737-300,4200,Среднемагистральный
1,Boeing 767-300,7900,Дальнемагистральный
0,Boeing 777-300,11100,Дальнемагистральный
8,Bombardier CRJ-200,2700,Среднемагистральный
7,Cessna 208 Caravan,1200,Ближнемагистральный
2,Sukhoi SuperJet-100,3000,Среднемагистральный


## Соединения

В тех случаях, когда информации, содержащейся в одной таблице, недостаточно для получения требуемого результата, используют __соединение (join)__ таблиц. Покажем способ выполнения соединения на примере следующего запроса: выбрать все места, предусмотренные компоновкой салона самолета Cessna 208 Caravan.

```postgresql
SELECT a.aircraft_code, a.model, s.seat_no, s.fare_conditions
  FROM seats AS s
  JOIN aircrafts AS a
    ON s.aircraft_code = a.aircraft_code
  WHERE a.model ~ '^Cessna'
  ORDER BY s.seat_no;
```

В предложении `WHERE` мы применили регулярное выражение, хотя в данном случае можно было с таким же успехом воспользоваться и оператором `LIKE` или функцией `substr`.

Данная команда иллюстрирует соединение двух таблиц на основе равенства значений атрибутов.

В этой команде в предложении `FROM` указаны две таблицы — `aircrafts` и `seats`, причем каждая из них получила еще и псевдоним с помощью ключевого слова `AS` (заметим, что оно не является обязательным). Конечно, псевдонимы могут состоять не только из одной буквы, как в нашем примере. Псевдонимы удобны в тех случаях, когда в соединяемых таблицах есть одноименные атрибуты. В таких случаях в списке атрибутов, следующих за ключевым словом `SELECT`, необходимо указывать либо имя таблицы, из которой выбирается значение этого атрибута, либо ее псевдоним, но псевдоним может быть коротким, что удобнее при написании команды. Псевдоним и атрибут соединяются символом «.». Псевдонимы используются и в предложениях `WHERE`, `GROUP BY`, `ORDER BY`, `HAVING`, т. е. во всех частях команды `SELECT`.

Если бы в качестве исходных сведений мы получили сразу код самолета — CN1, то запрос свелся бы к выборке из одной таблицы «Места». Он был бы таким:

```postgresql
SELECT * FROM seats WHERE aircraft_code = 'CN1';
```

In [5]:
query = """SELECT * FROM bookings.seats;"""

seats = pd.read_sql(query, conn)
seats.head()

,aircraft_code,seat_no,fare_conditions
0,319,2A,Business
1,319,2C,Business
2,319,2D,Business
3,319,2F,Business
4,319,3A,Business


Если подвести итог, то можно упрощенно объяснить механизм построения соединения следующим образом.
Сначала формируются все попарные комбинации строк из обеих таблиц, т. е. декартово произведение множеств строк этих таблиц. Эти комбинированные строки включают в себя все атрибуты обеих таблиц.

Затем в дело вступает условие `s.aircraft_code = a.aircraft_code`. Это означает, что в результирующем множестве строк останутся только те из них, в которых значения атрибута aircraft_code, взятые из таблицы `aircrafts` и из таблицы `seats`, одинаковые. Строки, не удовлетворяющие этому критерию, отфильтровываются.
Это означает на практике, что каждой строке из таблицы «Места» мы сопоставили только одну конкретную строку из таблицы «Самолеты», из которой мы теперь мо- жем взять значение атрибута «Модель самолета», чтобы включить ее в итоговый вывод данных.

На практике описанный механизм не реализуется буквально. Специальная подсистема PostgreSQL, называемая планировщиком, строит план выполнения запроса, который является гораздо более эффективным, чем упрощенный план, представленный здесь.

Запрос, который мы рассмотрели, можно записать немного по-другому, без использования предложения `JOIN` (обратите внимание, что мы не использовали ключевое слово `AS` для назначения псевдонимов таблицам).

```postgresql
SELECT a.aircraft_code, a.model, s.seat_no, s.fare_conditions
  FROM seats s, aircrafts a
  WHERE s.aircraft_code = a.aircraft_code
    AND a.model ~ '^Cessna'
  ORDER BY s.seat_no;
```

В этом варианте условие соединения таблиц `s.aircraft_code = a.aircraft_code` перешло из предложения `FROM` в предложение `WHERE`, а таблицы просто перечислены в предложении `FROM` через запятую. Простые запросы зачастую записывают именно в такой форме, без предложения `JOIN`, а в предложении `WHERE` указывают критерии, которым должны удовлетворять результирующие строки.

In [108]:
seats.merge(aircrafts[aircrafts['model'] \
                      .str.contains(r'^Cessna')],
                       on='aircraft_code')[['aircraft_code', 'model',
                                            'seat_no', 'fare_conditions']] \
                      .sort_values('seat_no')

,aircraft_code,model,seat_no,fare_conditions
0,CN1,Cessna 208 Caravan,1A,Economy
1,CN1,Cessna 208 Caravan,1B,Economy
2,CN1,Cessna 208 Caravan,2A,Economy
3,CN1,Cessna 208 Caravan,2B,Economy
4,CN1,Cessna 208 Caravan,3A,Economy
5,CN1,Cessna 208 Caravan,3B,Economy
6,CN1,Cessna 208 Caravan,4A,Economy
7,CN1,Cessna 208 Caravan,4B,Economy
8,CN1,Cessna 208 Caravan,5A,Economy
9,CN1,Cessna 208 Caravan,5B,Economy


> Изучая язык SQL вообще и способы выполнения соединений в частности, нужно иметь в виду, что результатом любых реляционных операций над отношениями (таблицами, представлениями) также является отношение. Поэтому такие операции можно произвольно комбинировать друг с другом.

Покажем три способа выполнения __соединения таблицы с самой собой__, отличающиеся синтаксически, но являющиеся функционально эквивалентными. Наш запрос-иллюстрация должен выяснить: сколько всего маршрутов нужно было бы сформировать, если бы требовалось соединить каждый город со всеми остальными городами? Если в городе имеется более одного аэропорта, то договоримся рейсы из каждого из них (в каждый из них) считать отдельными маршрутами. Поэтому правильнее было бы говорить не о маршрутах из каждого города, а о маршрутах из каждого аэропорта во все другие аэропорты. Конечно, рейсов из любого города в тот же самый город быть не должно.

Первый вариант запроса использует обычное перечисление имен таблиц в предложении `FROM`. Поскольку имена таблиц совпадают, используются псевдонимы. В таком случае СУБД обращается к таблице дважды, как если бы это были различные таблицы.

```postgresql
SELECT count( * )
  FROM airports a1, airports a2
  WHERE a1.city <> a2.city;
```


Как мы уже говорили ранее, СУБД соединяет каждую строку первой таблицы с каждой строкой второй таблицы, т. е. формирует декартово произведение таблиц — все попарные комбинации строк из двух таблиц. Затем СУБД отбрасывает те комбинированные строки, которые не удовлетворяют условию, приведенному в предложении `WHERE`. В нашем примере условие как раз и отражает требование о том, что рейсов из одного города в тот же самый город быть не должно.

Во втором варианте запроса мы используем соединение таблиц на основе неравенства значений атрибутов. Тем самым мы перенесли условие отбора результирующих строк из предложения `WHERE` в предложение `FROM`.

```postgresql
SELECT count( * )
  FROM airports a1
  JOIN airports a2 ON a1.city <> a2.city;
```

Третий вариант предусматривает явное использование декартова произведения таблиц. Для этого служит предложение `CROSS JOIN`. Лишние строки, как и в первом варианте, отсеиваем с помощью предложения `WHERE`:

```postgresql
SELECT count( * )
  FROM airports a1 CROSS JOIN airports a2
  WHERE a1.city <> a2.city;
```

С точки зрения СУБД эти три варианта эквивалентны и отличаются лишь синтаксисом. Для них PostgreSQL выберет один и тот же план (порядок) выполнения запроса.

В pandas получить декартово произведение датафрейма на него самого можно, смержив датафрейм с самим собой по стролбцу с произвольным ключем.

In [143]:
airports.assign(key=1) \
        .merge(airports.assign(key=1), on='key', how='outer') \
        .drop('key', axis=1).query('city_x != city_y').__len__()

10704

Теперь обратимся к так называемым внешним соединениям. Зададимся вопросом: сколько маршрутов обслуживают самолеты каждого типа? Если не требовать вывода наименований моделей самолетов, тогда всю необходимую информацию можно по лучить из материализованного представления «Маршруты» (routes). Но мы все же будем выводить и наименования моделей, поэтому обратимся также к таблице «Самолеты» (aircrafts). Соединим эти таблицы на основе атрибута `aircraft_code`, сгруппируем строки и просто воспользуемся функцией `count`. В этом запросе внешнее соединение еще не используется.

```postgresql
SELECT r.aircraft_code, a.model, count( * ) AS num_routes
  FROM routes r
  JOIN aircrafts a
    ON r.aircraft_code = a.aircraft_code
  GROUP BY 1, 2
  ORDER BY 3 DESC;
```

In [6]:
query = """SELECT * FROM bookings.routes;"""

routes = pd.read_sql(query, conn)
routes.head()

,flight_no,departure_airport,departure_airport_name,departure_city,arrival_airport,arrival_airport_name,arrival_city,aircraft_code,duration,days_of_week
0,PG0001,UIK,Усть-Илимск,Усть-Илимск,SGC,Сургут,Сургут,CR2,02:20:00,[1]
1,PG0002,SGC,Сургут,Сургут,UIK,Усть-Илимск,Усть-Илимск,CR2,02:20:00,[2]
2,PG0003,IWA,Иваново-Южный,Иваново,AER,Сочи,Сочи,CR2,02:10:00,"[1, 4]"
3,PG0004,AER,Сочи,Сочи,IWA,Иваново-Южный,Иваново,CR2,02:10:00,"[2, 5]"
4,PG0005,DME,Домодедово,Москва,PKV,Псков,Псков,CN1,02:05:00,"[2, 4, 7]"


In [161]:
aircrafts.merge(routes.assign(num_routes=1), on='aircraft_code') \
         .groupby(['aircraft_code', 'model']) \
         .agg({'num_routes': 'count'}) \
         .sort_values('num_routes', ascending=False)

,,num_routes
aircraft_code,model,
CR2,Bombardier CRJ-200,232
CN1,Cessna 208 Caravan,170
SU9,Sukhoi SuperJet-100,158
319,Airbus A319-100,46
733,Boeing 737-300,36
321,Airbus A321-200,32
763,Boeing 767-300,26
773,Boeing 777-300,10


Обратите внимание, что таблица «Самолеты» содержит 9 моделей, а в этой выборке лишь 8 строк. Значит, какая-то модель самолета не участвует в выполнении рейсов. Как ее выявить?

С помощью такого запроса:

```postgresql
SELECT a.aircraft_code AS a_code
       , a.model
       , r.aircraft_code AS r_code
       , COUNT( r.aircraft_code ) AS num_routes
  FROM aircrafts a
  LEFT OUTER JOIN routes r
    ON r.aircraft_code = a.aircraft_code
  GROUP BY 1, 2, 3
  ORDER BY 4 DESC;
```

В данном запросе используется левое внешнее соединение — об этом говорит предложение `LEFT OUTER JOIN`.

В качестве базовой таблицы выбирается таблица `aircrafts`, указанная в запросе слева от предложения `LEFT OUTER JOIN`, и для каждой строки, находящейся в ней, из таблицы `routes` подбираются строки, в которых значение атрибута `aircraft_code` такое же, как и в текущей строке таблицы `aircrafts`. Если в таблице `routes` нет ни одной соответствующей строки, то при отсутствии ключевых слов `LEFT OUTER` результирующая комбинированная строка просто не будет сформирована и не попадет в выборку. Но при наличии ключевых слов `LEFT OUTER` результирующая строка все равно будет сформирована.

Обратите внимание, что параметром функции `count` является столбец из таблицы `routes`, поэтому `count` и выдает число 0 для самолета с кодом 320. Если заменить его на одноименный столбец из таблицы `aircrafts`, тогда `count` выдаст 1, что будет противоречить цели нашей задачи — подсчитать число рейсов, выполняемых на самолетах каждого типа. Напомним, что если функция `count` в качестве параметра получает не символ «∗», а имя столбца, тогда она подсчитывает число строк, в которых значение в этом столбце определено (не равно NULL).

In [172]:
aircrafts.merge(routes.assign(num_routes=1), on='aircraft_code', how='outer') \
         .groupby(['aircraft_code', 'model'])[['num_routes']].count() \
         .sort_values('num_routes', ascending=False)

,,num_routes
aircraft_code,model,
CR2,Bombardier CRJ-200,232
CN1,Cessna 208 Caravan,170
SU9,Sukhoi SuperJet-100,158
319,Airbus A319-100,46
733,Boeing 737-300,36
321,Airbus A321-200,32
763,Boeing 767-300,26
773,Boeing 777-300,10
320,Airbus A320-200,0


Кроме левого внешнего соединения существует также и правое внешнее соединение — RIGHT OUTER JOIN.

В этом случае в качестве базовой выбирается таблица, имя которой указано справа от предложения `RIGHT OUTER JOIN`, а механизм получения результирующих строк в случае, когда для строки базовой таблицы не находится пары во второй таблице, точно такой же, как и для левого внешнего соединения. Как сказано в документации, правое внешнее соединение является лишь синтаксическим приемом, поскольку всегда можно заменить его левым внешним соединением, поменяв при этом имена таблиц местами.

Важно учитывать, что порядок следования таблиц в предложениях `LEFT (RIGHT) OUTER JOIN` никак не влияет на порядок столбцов в предложении `SELECT`. В вышеприведенном запросе мы написали

```postgresql
SELECT a.aircraft_code AS a_code,
  a.model,
  r.aircraft_code AS r_code, 
  ...
```
Но если бы нам это было нужно, то мы могли бы поменять столбцы местами:

```postgresql
SELECT r.aircraft_code AS r_code,
  a.model,
  a.aircraft_code AS a_code,
  ...
```

Комбинацией этих двух видов внешних соединений является полное внешнее соединение — FULL OUTER JOIN.
В этом случае в выборку включаются строки из левой таблицы, для которых не нашлось соответствующих строк в правой таблице, и строки из правой таблицы, для которых не нашлось соответствующих строк в левой таблице.

В практической работе при выполнении выборок зачастую выполняются многотабличные запросы, включающие три таблицы и более. В качестве примера рассмотрим такую задачу: определить число пассажиров, не пришедших на регистрацию билетов и, следовательно, не вылетевших в пункт назначения. Будем учитывать только рейсы, у которых фактическое время вылета не пустое, т. е. рейсы, имеющие статус Departed или Arrived.

```postgresql
SELECT count( * )
  FROM ( ticket_flights t
         JOIN flights f ON t.flight_id = f.flight_id
       )
  LEFT OUTER JOIN boarding_passes b
    ON t.ticket_no = b.ticket_no AND t.flight_id = b.flight_id
  WHERE f.actual_departure IS NOT NULL AND b.flight_id IS NULL;
```

При формировании запроса надо вспомнить, что таблица «Посадочные талоны» (`boarding_passes`) связана с таблицей «Перелеты» (`ticket_flights`) по внешнему ключу, а тип связи — 1:1, т. е. каждой строке из таблицы `ticket_flights` соответствует не более одной строки в таблице `boarding_passes`: ведь строка в таблицу `boarding_passes` добавляется только тогда, когда пассажир прошел регистрацию на рейс. Однако теоретически, да и практически тоже, пассажир может на регистрацию не явиться, тогда строка в таблицу `boarding_passes` добавлена не будет.

Поскольку нас интересуют только рейсы с непустым временем вылета, нам придется обратиться к таблице «Рейсы» (`flights`) и соединить ее с таблицей `ticket_flights` по атрибуту `flight_id`. А затем для подключения таблицы `boarding_passes` мы используем левое внешнее соединение, т. к. в этой таблице может не оказаться строки, соответствующей строке из таблицы `ticket_flights`.

В предложении `WHERE` второе условие — `b.flight_id IS NULL`. Оно и позволяет выявить те комбинированные строки, в которых столбцам таблицы `boarding_passes` были назначены значения NULL из-за того, что в ней не нашлось строки, для которой выполнялось бы условие `t.ticket_no = b.ticket_no AND t.flight_id = b.flight_id`. Конечно, для проверки на NULL мы могли использовать любой столбец таблицы `boarding_passes`, а не только `b.flight_id`.

In [7]:
query = """SELECT * FROM bookings.ticket_flights;"""

ticket_flights = pd.read_sql(query, conn)
ticket_flights.head()

,ticket_no,flight_id,fare_conditions,amount
0,0005432159776,30625,Business,42100.0
1,0005435212351,30625,Business,42100.0
2,0005435212386,30625,Business,42100.0
3,0005435212381,30625,Business,42100.0
4,0005432211370,30625,Business,42100.0


In [8]:
query = """SELECT * FROM bookings.flights;"""

flights = pd.read_sql(query, conn)
flights.head()

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,1,PG0405,2016-09-13 05:35:00+00:00,2016-09-13 06:30:00+00:00,DME,LED,Arrived,321,2016-09-13 05:44:00+00:00,2016-09-13 06:39:00+00:00
1,2,PG0404,2016-10-03 15:05:00+00:00,2016-10-03 16:00:00+00:00,DME,LED,Arrived,321,2016-10-03 15:06:00+00:00,2016-10-03 16:01:00+00:00
2,3,PG0405,2016-10-03 05:35:00+00:00,2016-10-03 06:30:00+00:00,DME,LED,Arrived,321,2016-10-03 05:39:00+00:00,2016-10-03 06:34:00+00:00
3,4,PG0402,2016-11-07 08:25:00+00:00,2016-11-07 09:20:00+00:00,DME,LED,Scheduled,321,NaT,NaT
4,5,PG0405,2016-10-14 05:35:00+00:00,2016-10-14 06:30:00+00:00,DME,LED,On Time,321,NaT,NaT


In [9]:
query = """SELECT * FROM bookings.boarding_passes;"""

boarding_passes = pd.read_sql(query, conn)
boarding_passes.head()

,ticket_no,flight_id,boarding_no,seat_no
0,0005435212351,30625,1,2D
1,0005435212386,30625,2,3G
2,0005435212381,30625,3,4H
3,0005432211370,30625,4,5D
4,0005435212357,30625,5,11A


In [201]:
not_boarded = ticket_flights.merge(flights, on='flight_id') \
                            .merge(boarding_passes, how='outer', on=['ticket_no','flight_id'])

not_boarded[(~not_boarded['actual_departure'].isnull()) & (not_boarded['flight_id'].isnull())].__len__()

0

Оказывается, таких пассажиров нет.

Теперь рассмотрим более сложный пример. Известно, что в компьютерных системах бывают сбои. Предположим, что возможна такая ситуация: при бронировании билета пассажир выбрал один класс обслуживания, например, Business, а при регистрации на рейс ему выдали посадочный талон на то место в салоне самолета, где класс обслуживания — Economy. Необходимо выявить все случаи несовпадения классов обслуживания.

Сведения о классе обслуживания, который пассажир выбрал при бронировании билета, содержатся в таблице «Перелеты» (`ticket_flights`). Однако в таблице «Посадочные талоны» (`boarding_passes`), которая «отвечает» за посадку на рейс, сведений о классе обслуживания, который пассажир получил при регистрации, нет. Эти сведения можно получить только из таблицы «Места» (`seats`). Причем сделать это можно, зная код модели самолета, выполняющего рейс, и номер места в салоне самолета. Номер места можно взять из таблицы `boarding_passes`, а код модели самолета можно получить из таблицы «Рейсы» (`flights`), связав ее с таблицей `boarding_passes`.

Для полноты информационной картины необходимо получить еще фамилию и имя пассажира из таблицы «Билеты» (`tickets`), связав ее с таблицей `ticket_flights` по атрибуту «Номер билета» (`ticket_no`). При формировании запроса выберем в качестве, условно говоря, базовой таблицы таблицу `boarding_passes`, а затем будем поэтапно подключать остальные таблицы. В предложении `WHERE` будет только одно условие: несовпадение требуемого и фактического классов обслуживания.

В результате получим запрос, включающий пять таблиц. Он не выдаст ни одной строки, значит, пассажиров, получивших неправильный класс обслуживания, не было.

```postgresql
SELECT f.flight_no
       , f.scheduled_departure
       , f.flight_id
       , f.departure_airport
       , f.arrival_airport
       , f.aircraft_code
       , t.passenger_name
       , tf.fare_conditions AS fc_to_be
       , s.fare_conditions AS fc_fact
       , b.seat_no
  FROM boarding_passes b
  JOIN ticket_flights tf
    ON b.ticket_no = tf.ticket_no AND b.flight_id = tf.flight_id
  JOIN tickets t ON tf.ticket_no = t.ticket_no
  JOIN flights f ON tf.flight_id = f.flight_id
  JOIN seats s
   ON b.seat_no = s.seat_no AND f.aircraft_code = s.aircraft_code
  WHERE tf.fare_conditions <> s.fare_conditions
  ORDER BY f.flight_no, f.scheduled_departure;
```

In [10]:
query = """SELECT * FROM bookings.tickets;"""

tickets = pd.read_sql(query, conn)
tickets.head()

,ticket_no,book_ref,passenger_id,passenger_name,contact_data
0,0005432000987,06B046,8149 604011,VALERIY TIKHONOV,{'phone': '+70127117011'}
1,0005432000988,06B046,8499 420203,EVGENIYA ALEKSEEVA,{'phone': '+70378089255'}
2,0005432000989,E170C3,1011 752484,ARTUR GERASIMOV,{'phone': '+70760429203'}
3,0005432000990,E170C3,4849 400049,ALINA VOLKOVA,{'email': 'volkova.alina_03101973@postgrespro....
4,0005432000991,F313DD,6615 976589,MAKSIM ZHUKOV,"{'email': 'm-zhukov061972@postgrespro.ru', 'ph..."


In [18]:
boarding_passes.merge(ticket_flights, on=['ticket_no', 'flight_id']) \
               .merge(tickets, on='ticket_no') \
               .merge(flights, on='flight_id') \
               .merge(seats, on=['seat_no' ,'aircraft_code']) \
               .query('fare_conditions_x != fare_conditions_y').__len__()

0

Чтобы все же удостовериться в работоспособности этого запроса, можно в таблице `boarding_passes` изменить в одной строке номер места таким образом, чтобы этот пассажир переместился из салона экономического класса в салон бизнес-класса.

```postgresql
UPDATE boarding_passes
  SET seat_no = '1A'
  WHERE flight_id = 1 AND seat_no = '17A';
```

In [27]:
boarding_passes.loc[(boarding_passes.seat_no == '17A') & (boarding_passes.flight_id == 1), 'seat_no'] = '1A'

Выполним запрос еще раз, и теперь он выдаст одну строку.

In [28]:
boarding_passes.merge(ticket_flights, on=['ticket_no', 'flight_id']) \
               .merge(tickets, on='ticket_no') \
               .merge(flights, on='flight_id') \
               .merge(seats, on=['seat_no' ,'aircraft_code']) \
               .query('fare_conditions_x != fare_conditions_y').__len__()

1

В предложении `FROM` можно использовать виртуальные таблицы, сформированные с помощью ключевого слова `VALUES`. Предположим, что для выработки финансовой стратегии нашей авиакомпании требуется распределение количества бронирований по диапазонам сумм с шагом в 100 тысяч рублей. Максимальная сумма в одном бронировании составляет 1 204 500 рублей. Учтем это при формировании диапазонов.

Виртуальной таблице, создаваемой с помощью ключевого слова `VALUES`, присваивают имя с помощью ключевого слова `AS`. После имени в круглых скобках приводится список имен столбцов этой таблицы.

```postgresql
SELECT r.min_sum, r.max_sum, count( b.* )
  FROM bookings b
  RIGHT OUTER JOIN
    ( VALUES (       0,  100000 ), (  100000,  200000 ),
             (  200000,  300000 ), (  300000,  400000 ),
             (  400000,  500000 ), (  500000,  600000 ),
             (  600000,  700000 ), (  700000,  800000 ),
             (  800000,  900000 ), (  900000, 1000000 ),
             ( 1000000, 1100000 ), ( 1100000, 1200000 ),
             ( 1200000, 1300000 )
     ) AS r ( min_sum, max_sum )
     ON b.total_amount >= r.min_sum AND b.total_amount < r.max_sum
  GROUP BY r.min_sum, r.max_sum
  ORDER BY r.min_sum;  
 ```
 
В этом запросе мы использовали внешнее соединение. Сделано это для того, чтобы в случаях, когда в каком-то диапазоне не окажется ни одного бронирования, результирующая строка выборки все же была бы сформирована. А правое соединение было выбрано только потому, что в качестве первой, базовой, таблицы мы выбрали таблицу «Бронирования» (`bookings`), но именно в ней может не оказаться ни одной строки для соединения с какой-либо строкой виртуальной таблицы. А все строки виртуальной таблицы, стоящей справа от предложения `RIGHT OUTER JOIN`, должны быть обязательно представлены в выборке: это позволит сразу увидеть «пустые» диапазоны, если они будут.

Можно было использовать и левое внешнее соединение, поменяв таблицы местами.

In [33]:
query = """SELECT * FROM bookings.bookings;"""

bookings = pd.read_sql(query, conn)
bookings.head()

,book_ref,book_date,total_amount
0,00000F,2016-09-01 23:12:00+00:00,265700.0
1,000012,2016-09-11 05:02:00+00:00,37900.0
2,000068,2016-10-13 10:27:00+00:00,18100.0
3,000181,2016-10-08 09:28:00+00:00,131800.0
4,0002D8,2016-10-05 17:40:00+00:00,23600.0


In [91]:
bookings.groupby(pd.cut(bookings.total_amount, bins=range(0, 1400000, 100000), right=False)).size()

total_amount
[0, 100000)           198314
[100000, 200000)       46943
[200000, 300000)       11916
[300000, 400000)        3260
[400000, 500000)        1357
[500000, 600000)         681
[600000, 700000)         222
[700000, 800000)          55
[800000, 900000)          24
[900000, 1000000)         11
[1000000, 1100000)         4
[1100000, 1200000)         0
[1200000, 1300000)         1
dtype: int64

В команде `SELECT` предусмотрены средства для выполнения операций с выборками, как с множествами, а именно:

 - `UNION` для вычисления объединения множеств строк из двух выборок;
 - `INTERSECT` для вычисления пересечения множеств строк из двух выборок;
 - `EXCEPT` для вычисления разности множеств строк из двух выборок.

Запросы должны возвращать одинаковое число столбцов, типы данных у столбцов также должны совпадать.
Рассмотрим эти операции, используя материализованное представление «Маршруты» (`routes`).

Начнем с операции объединения множеств строк — `UNION`. Строка включается в итоговое множество (выборку), если она присутствует хотя бы в одном из них. Строки-дубликаты в результирующее множество не включаются. Для их включения нужно использовать `UNION ALL`.

Вопрос: в какие города можно улететь либо из Москвы, либо из Санкт-Петербурга?

```postgresql
SELECT arrival_city FROM routes
  WHERE departure_city = 'Москва'
UNION
SELECT arrival_city FROM routes
  WHERE departure_city = 'Санкт-Петербург'
ORDER BY arrival_city;
```

In [104]:
pd.Series(routes[routes.departure_city.isin(['Москва','Санкт-Петербург'])]['arrival_city'].unique()).sort_values()

82            Абакан
65           Анадырь
64             Анапа
69       Архангельск
31         Астрахань
           ...      
18         Череповец
68            Элиста
43    Южно-Сахалинск
62            Якутск
77         Ярославль
Length: 87, dtype: object

Рассмотрим операцию пересечения множеств строк — `INTERSECT`. Строка включается в итоговое множество (выборку), если она присутствует в каждом из них. Строки-дубликаты в результирующее множество не включаются. Для их включения нужно использовать `INTERSECT ALL`.

Вопрос: в какие города можно улететь как из Москвы, так и из Санкт-Петербурга?

```postgresql
SELECT arrival_city FROM routes
  WHERE departure_city = 'Москва'
INTERSECT
SELECT arrival_city FROM routes
  WHERE departure_city = 'Санкт-Петербург'
ORDER BY arrival_city;
```

In [123]:
pd.Series(np.intersect1d(routes[routes.departure_city == 'Москва'].arrival_city,
                         routes[routes.departure_city == 'Санкт-Петербург'].arrival_city))

0             Воркута
1             Воронеж
2              Казань
3               Киров
4            Нерюнгри
5       Нижневартовск
6     Нижний Новгород
7            Ноябрьск
8            Оренбург
9           Советский
10             Тамбов
11             Тюмень
12          Хабаровск
13          Чебоксары
14             Элиста
dtype: object

В завершение рассмотрим операцию разности множеств строк — `EXCEPT`. Строка включается в итоговое множество (выборку), если она присутствует в первом множестве (выборке), но отсутствует во втором. Строки-дубликаты в результирующее множество не включаются. Для их включения нужно использовать `EXCEPT ALL`.

Вопрос: в какие города можно улететь из Санкт-Петербурга, но нельзя из Москвы?

```postgresql
SELECT arrival_city FROM routes
  WHERE departure_city = 'Санкт-Петербург'
EXCEPT
SELECT arrival_city FROM routes
  WHERE departure_city = 'Москва'
ORDER BY arrival_city;
```

Конечно, при выполнении этих операций можно соединять не только две таблицы, но и большее их число. При этом нужно либо учитывать приоритеты выполнения операций, либо использовать скобки. Согласно документации `INTERSECT` связывает свои подзапросы сильнее, чем `UNION`, а `EXCEPT` связывает свои подзапросы так же сильно, как `UNION`.


In [127]:
pd.Series(np.setdiff1d(routes[routes.departure_city == 'Санкт-Петербург'].arrival_city,
                       routes[routes.departure_city == 'Москва'].arrival_city))

0      Иркутск
1       Калуга
2       Москва
3      Удачный
4    Череповец
5       Якутск
6    Ярославль
dtype: object

## Агрегирование и группировка

Среди множества функций, имеющихся в PostgreSQL, важное место занимают агрегатные функции. С одной из них, функцией count, мы уже работали довольно много. Давайте рассмотрим еще ряд функций из этой группы и сделаем это на примере таблицы «Бронирования».

Для расчета среднего значения по столбцу используется функция `avg` (от слова average).

```postgresql
SELECT AVG( total_amount ) FROM bookings;
```

In [128]:
bookings.total_amount.mean()

79025.60581152869

Для получения максимального значения по столбцу используется функция `max`.

```postgresql
SELECT MAX( total_amount ) FROM bookings;
```

In [129]:
bookings.total_amount.max()

1204500.0

Для получения минимального значения по столбцу используется функция `min`. 

```postgresql
SELECT MIN( total_amount ) FROM bookings;
```

In [130]:
bookings.total_amount.min()

3400.0

Мы уже много раз выполняли группировку строк в выборке при помощи предложения `GROUP BY`, поэтому рассмотрим только два примера.

Первый будет таким: давайте подсчитаем, сколько маршрутов предусмотрено из Москвы в другие города. При формировании запроса не будем учитывать частоту рейсов в неделю, т. е. независимо от того, выполняется какой-то рейс один раз в неделю или семь раз, он учитывается только однократно. Воспользуемся материализованным представлением «Маршруты».

```postgresql
SELECT arrival_city, count( * )
  FROM routes
  WHERE departure_city = 'Москва'
  GROUP BY arrival_city
  ORDER BY count DESC;
```

In [142]:
routes.query('departure_city == "Москва"') \
      .groupby('arrival_city') \
      .size() \
      .sort_values(ascending=False)

arrival_city
Санкт-Петербург     12
Брянск               9
Ульяновск            5
Петрозаводск         4
Йошкар-Ола           4
                    ..
Минеральные Воды     1
Магнитогорск         1
Липецк               1
Курск                1
Абакан               1
Length: 80, dtype: int64

В качестве второго примера рассмотрим ситуацию, когда руководству компании потребовалась обобщенная информация по частоте выполнения рейсов, а именно: сколько рейсов выполняется ежедневно, сколько рейсов — шесть дней в неделю, пять и т. д. Опять обратимся к материализованному представлению «Маршруты». Но теперь при формировании запроса, в отличие от первого примера, воспользуемся столбцом `days_of_week`, в котором содержатся массивы номеров дней недели, когда выполняется данный рейс.

```postgresql
SELECT array_length( days_of_week, 1 ) AS days_per_week
       , count( * ) AS num_routes
  FROM routes
  GROUP BY days_per_week
  ORDER BY 1 desc;
```

В этом запросе используется функция `array_length`, возвращающая количество элементов в указанном измерении массива. Поскольку массив одномерный, то вторым параметром функции будет число 1 — первое измерение.
При выполнении выборок можно с помощью условий, заданных в предложении `WHERE`, сузить множество выбираемых строк. 

In [149]:
routes.groupby(routes.days_of_week.apply(len)).size().sort_index(ascending=False)

days_of_week
7    482
3     54
2     88
1     86
dtype: int64

Аналогичная возможность существует и при выполнении группировок: можно включить в результирующее множество не все строки, а лишь те, которые удовлетворяют некоторому условию. Это условие можно задать в предложении `HAVING`. Важно помнить, что предложение `WHERE` работает с отдельными строками еще до выполнения группировки с помощью `GROUP BY`, а предложение `HAVING` — уже после выполнения группировки.

В качестве примера приведем такой запрос: определить, сколько существует маршрутов из каждого города в другие города, и вывести названия городов, из которых в другие города существует не менее 15 маршрутов.

```postgresql
SELECT departure_city, count( * ) FROM routes
  GROUP BY departure_city
  HAVING count( * ) >= 15
  ORDER BY count DESC;
```

In [176]:
routes.groupby('departure_city').filter(lambda x: len(x) >= 15) \
                                .groupby('departure_city') \
                                .size().sort_values(ascending=False)

departure_city
Москва             154
Санкт-Петербург     35
Новосибирск         19
Екатеринбург        15
dtype: int64

В подавляющем большинстве городов только один аэропорт, но есть и такие города, в которых более одного аэропорта. Давайте их выявим.

```postgresql
SELECT city, count( * )
  FROM airports
  GROUP BY city
  HAVING count( * ) > 1;
```

In [177]:
airports.groupby('city').filter(lambda x: len(x) > 1) \
                        .groupby('city').size()

city
Москва       3
Ульяновск    2
dtype: int64

Кроме обычных агрегатных функций существуют и так называемые оконные функции (window functions). Эти функции предоставляют возможность производить вычисления на множестве строк, логически связанных с текущей строкой, т. е. имеющих то или иное отношение к ней.
При работе с оконными функциями используются концепции *раздела* (partition) и *оконного кадра* (window frame). Сначала объясним эти понятия на примере.

Предположим, что руководство нашей компании хочет усовершенствовать тарифную политику и с этой целью просит нас предоставить сведения о распределении количества проданных билетов на некоторые рейсы во времени. Количество проданных билетов должно выводиться в виде накопленного показателя, суммирование должно производиться в пределах каждого календарного месяца.

Более детально, в столбцах `book_ref` и `book_date` выборки должны приводиться но- мер и время бронирования соответственно. В столбцах month и day должны указываться порядковый номер месяца и день этого месяца. Столбец count должен содержать суммарные (накопленные) количества билетов, проданных на каждый момент времени. С первого дня нового месяца подсчет числа проданных билетов начинается сначала.

Таким образом, в нашем примере в качестве раздела (partition) будет выступать множество строк, у которых даты продажи билета (т. е. даты бронирования) относятся к одному и тому же месяцу.

Понятие оконного кадра (window frame) является важным, поскольку многие оконные функции работают не со всеми строками раздела, а только с теми, которые образуют оконный кадр текущей строки. Если строки в разделе не упорядочены, то оконным кадром текущей строки по умолчанию считается множество всех строк раздела. Однако в том случае, когда строки в разделе упорядочены по какому-то критерию, тогда в состав оконного кадра по умолчанию включаются строки, начиная с первой строки раздела и заканчивая текущей строкой. Если же существуют строки, имеющие такое же значение критерия сортировки, что и текущая строка, и расположенные после нее, то они также включаются в состав оконного кадра текущей строки.

Теперь посмотрим, с помощью какого запроса был получен этот результат, и на его примере объясним синтаксические конструкции, используемые для работы с оконными функциями.

```postgresql
SELECT b.book_ref
       , b.book_date
       , extract( 'month' from b.book_date ) AS month
       , extract( 'day' from b.book_date ) AS day
       , count( * ) OVER (
         PARTITION BY date_trunc( 'month', b.book_date )
         ORDER BY b.book_date 
       ) AS count
  FROM ticket_flights tf
  JOIN tickets t ON tf.ticket_no = t.ticket_no
  JOIN bookings b ON t.book_ref = b.book_ref
  WHERE tf.flight_id = 1
  ORDER BY b.book_date;
```

Рассмотрим конструкцию, предназначенную для вызова оконной функции:

```postgresql
count( * ) OVER (
  PARTITION BY date_trunc( 'month', b.book_date )
  ORDER BY b.book_date
) AS count
```

В этой конструкции обязательным является ключевое слово `OVER`. Функция `count` — это обычная агрегатная функция, но если вслед за ней идет это ключевое слово, то она становится оконной функцией. Предложение `PARTITION BY` задает правило разбиения строк выборки на разделы. Предложение `ORDER BY` предписывает порядок сортировки строк в разделах.

Обобщая приведенные объяснения, можно сказать, что раздел включает в себя все строки выборки, имеющие в некотором смысле одинаковые свойства, например, одинаковые значения определенных выражений, задаваемых с помощью предложе- ния `PARTITION BY`. Это могут быть выражения, построенные на основе одного или нескольких столбцов таблицы (или таблиц, участвующих в соединении).

Оконный кадр состоит из подмножества строк данного раздела и привязан к текущей строке. Для определения границ кадра важным является наличие предложения `ORDER BY` при формировании раздела. В рассмотренном примере границы оконного кадра определялись по умолчанию. Однако для указания этих границ предусмотрены различные способы. Не только функция `count`, но и другие агрегатные функции (например, `sum`, `avg`) тоже могут применяться в качестве оконных функций.

Оконные функции, в отличие от обычных агрегатных функций, не требуют группировки строк, а работают на уровне отдельных (несгруппированных) строк. Однако если в запросе присутствуют предложения `GROUP BY` и `HAVING`, тогда оконные функции вызываются уже после них. В таком случае оконные функции будут работать со строками, являющимися результатом группировки.

In [252]:
frame = ticket_flights.merge(tickets, on='ticket_no').merge(bookings, on='book_ref').query('flight_id == 1')

In [253]:
frame.assign(count=frame.sort_values('book_date') \
            .groupby([frame.book_date.dt.month, frame.book_date.dt.day]) \
            .cumcount()+1).sort_values('book_date')

,ticket_no,flight_id,fare_conditions,amount,book_ref,passenger_id,passenger_name,contact_data,book_date,total_amount,count
14352,0005432816964,1,Economy,6700.0,A60039,6011 694032,TAISA KOROLEVA,"{'email': 't_koroleva.1964@postgrespro.ru', 'p...",2016-08-22 04:02:00+00:00,35200.0,1
14348,0005432816946,1,Economy,6700.0,554340,6137 327628,VALENTINA KOLESNIKOVA,{'email': 'kolesnikova.v21121973@postgrespro.r...,2016-08-23 15:04:00+00:00,36200.0,1
14245,0005432261100,1,Business,20000.0,854C4C,1511 328252,MARIYA KOVALEVA,"{'email': 'kovaleva-m-1973@postgrespro.ru', 'p...",2016-08-24 02:52:00+00:00,46700.0,1
14246,0005432261101,1,Business,20000.0,854C4C,8910 452385,YULIYA GAVRILOVA,{'email': 'yuliyagavrilova1972@postgrespro.ru'...,2016-08-24 02:52:00+00:00,46700.0,2
14247,0005432261102,1,Economy,6700.0,854C4C,7641 575183,NINA OVCHINNIKOVA,{'email': 'n_ovchinnikova.08071967@postgrespro...,2016-08-24 02:52:00+00:00,46700.0,3
...,...,...,...,...,...,...,...,...,...,...,...
14270,0005432261096,1,Business,20000.0,19FE38,9141 251461,ANNA ISAEVA,{'phone': '+70279389741'},2016-09-03 09:42:00+00:00,26700.0,2
14271,0005432261095,1,Economy,6700.0,19FE38,5660 835749,VALENTINA DMITRIEVA,{'phone': '+70167182132'},2016-09-03 09:42:00+00:00,26700.0,3
14343,0005432288928,1,Economy,6700.0,536A3D,1694 654851,NADEZHDA BARANOVA,{'email': 'baranova.n.20081974@postgrespro.ru'...,2016-09-03 11:19:00+00:00,35200.0,4
14341,0005432288927,1,Economy,6700.0,536A3D,3011 274373,OLGA KOMAROVA,{'phone': '+70995550419'},2016-09-03 11:19:00+00:00,35200.0,5


Рассмотрим еще один пример. Покажем, как с помощью оконной функции `rank` можно проранжировать аэропорты в пределах каждого часового пояса на основе их географической широты. Причем будем присваивать более высокий ранг тому аэропорту, который находится севернее.

```postgresql
SELECT airport_name
       , city
       , round( latitude::numeric, 2 ) AS ltd
       , timezone
       , rank() OVER (
           PARTITION BY timezone
           ORDER BY latitude DESC
       )
  FROM airports
  WHERE timezone IN ( 'Asia/Irkutsk', 'Asia/Krasnoyarsk' )
  ORDER BY timezone, rank;
```


В этом запросе в предложении `OVER ( PARTITION BY timezone ... )` указывается, что строки относятся к одному разделу на основе совпадения значений в столбце `timezone`. Обратите внимание, что хотя в предложении `OVER` задан порядок сортировки, действующий в пределах каждого окна, тем не менее, с помощью предложения `ORDER BY` указан также и порядок сортировки на уровне всего запроса.

In [293]:
asia_airports = airports[airports.timezone.isin(['Asia/Irkutsk', 'Asia/Krasnoyarsk'])].sort_values('latitude')

In [285]:
asia_airports['rank'] = asia_airports.groupby('timezone')['latitude'] \
                                     .rank(method="first", ascending=False)

In [287]:
asia_airports.sort_values(['timezone', 'rank'])

,airport_code,airport_name,city,longitude,latitude,timezone,rank
45,UIK,Усть-Илимск,Усть-Илимск,102.566000,58.135000,Asia/Irkutsk,1.0
16,UKX,Усть-Кут,Усть-Кут,105.716700,56.850000,Asia/Irkutsk,2.0
35,BTK,Братск,Братск,101.698331,56.370556,Asia/Irkutsk,3.0
32,IKT,Иркутск,Иркутск,104.388975,52.268028,Asia/Irkutsk,4.0
39,UUD,Байкал,Улан-Удэ,107.437644,51.807764,Asia/Irkutsk,5.0
99,NSK,Норильск,Норильск,87.332183,69.311053,Asia/Krasnoyarsk,1.0
23,SWT,Стрежевой,Стрежевой,77.650000,60.716667,Asia/Krasnoyarsk,2.0
96,TOF,Богашёво,Томск,85.208333,56.380278,Asia/Krasnoyarsk,3.0
20,KJA,Емельяново,Красноярск,92.475000,56.180000,Asia/Krasnoyarsk,4.0
36,ABA,Абакан,Абакан,91.385000,53.740000,Asia/Krasnoyarsk,5.0


## Подзапросы